In [1]:
import dask.dataframe as dd
import dask.array as da
import os
import csv
import numba
import numpy as np
from tqdm import tqdm
import pandas as pd
import dask
import xarray as xr
import scipy.io as sio
from skimage import exposure
from scipy.interpolate import interp1d

In [2]:
solar_path = 'Aug14SolarRef.dat'
HySICS_wl_path = 'HySICS files/WLHysics.sav'

HySICS_LW_data_path = 'Desert_vegetation_clouds/'
LW_datacube_path = 'data_cube_water.npy'
LW_rgb_path = 'rgb_water.npy'
LRT_LW_path = 'WC files/'
verbose_LW_path = 'WC files/verbose/'

HySICS_IC_data_path = 'Thick_clouds1/'
IC_datacube_path = 'data_cube_ice.npy'
IC_rgb_path = 'rgb_ice.npy'
LRT_IC_path = 'IC files/ghm/'
verbose_IC_path = 'IC files/ghm/verbose_ghm/'

water_dict = {'solar_path':solar_path,'HySICS_wl_path':HySICS_wl_path,'HySICS_data_path':HySICS_LW_data_path, \
              'LRT_path':LRT_LW_path,'LRT_verbose_path':verbose_LW_path, 'phase':'Liquid Water', \
             'data_cube_path':LW_datacube_path,'rgb_path':LW_rgb_path}
ice_dict = {'solar_path':solar_path,'HySICS_wl_path':HySICS_wl_path,'HySICS_data_path':HySICS_IC_data_path, \
              'LRT_path':LRT_IC_path,'LRT_verbose_path':verbose_IC_path, 'phase':'Ice',\
           'data_cube_path':IC_datacube_path,'rgb_path':IC_rgb_path}

In [3]:
phase_dict = water_dict

In [4]:
def read_HySICS_wl(phase_dict):
    hysics_wl = sio.readsav(phase_dict['HySICS_wl_path'])
    hysics_wl = hysics_wl['wlsample']
    hysics_wl = [x+3.5 for x in hysics_wl]
    return hysics_wl;

@dask.delayed
def read_df(file):
    try:
        array = np.genfromtxt(file,delimiter=',')
        return array
    except:
        pass

def generate_datacube(phase_dict,wl_list):
    path = phase_dict['HySICS_data_path']
    files = [os.path.join(path,f) for f in os.listdir(path) if f.startswith('img')]
    files.sort()

    dfs = [read_df(file) for file in files]

    sample = dfs[0].compute()
    das = [da.from_delayed(item, shape=sample.shape, dtype=sample.dtype) for item in dfs]
    array = da.stack(das)

    datacube = xr.DataArray(array, dims=['x', 'y', 'wavelength'], name='datacube', coords={'wavelength':wl_list})
    return datacube;

In [5]:
wl_list = read_HySICS_wl(phase_dict)
datacube_dvc = generate_datacube(phase_dict,wl_list)
datacube_dvc.to_netcdf('datacube_dvc.nc',format='NETCDF4')

In [6]:
def two_percent_linear_stretch(x,idx):
    c = x.isel(wavelength=idx)/np.amax(x.isel(wavelength=idx))
    p2, p98 = np.percentile(c, (2, 98))
    c_scaled = exposure.rescale_intensity(c, in_range=(p2, p98))
    return c_scaled;


def find_rgb_idx(wl_list):
    r_idx = np.argmin([np.abs(670 - x) for x in wl_list])
    g_idx = np.argmin([np.abs(555 - x) for x in wl_list])
    b_idx = np.argmin([np.abs(443 - x) for x in wl_list])
    rgb_idx_list = [r_idx,g_idx,b_idx]
    return (rgb_idx_list); 


def apply_stretch(datacube):
    rgb_idx_list = find_rgb_idx(datacube.coords['wavelength'])
    
    red = two_percent_linear_stretch(datacube,rgb_idx_list[0])
    green = two_percent_linear_stretch(datacube,rgb_idx_list[1])
    blue = two_percent_linear_stretch(datacube,rgb_idx_list[2])

    rgb = np.dstack((red,green,blue))
    rgb = xr.DataArray(rgb,dims=('x','y','rgb'))
    return rgb;   

rgb = apply_stretch(datacube_dvc)
rgb.to_netcdf('rgb_dvc.nc',format='NETCDF4')

In [7]:
datacube_dvc = xr.open_dataarray('datacube_dvc.nc',chunks=10)
rgb = xr.open_dataarray('rgb_dvc.nc',chunks=10)

In [8]:
def read_verbose(verbose_file,phase):
    data = [x for x in open(verbose_file,'r')]
    a = data.index('Using new intensity correction, with phase functions\n')
    n=4 if phase=='Liquid Water' else 5
    tau = data[a:][67].split('|')[n].split()[0]
    return(tau);

def read_LRT(file,phase):
    fs = file.split('_')
    reff = fs[3][1:] if phase =='Liquid Water' else fs[2][1:]
    
    fv = file.split('/')
    ff = fv[1][0:-4] if phase =='Liquid Water' else fv[1][0:-9]
    verbose_file = fv[0]+'/verbose/'+ff+'_550nm_verbose.txt'
    tau = read_verbose(verbose_file,phase)
    
    data = [x for x in csv.reader(open(file,'r'),delimiter='\t')]  
    data = data[5:]
    wl=[]           #Wavelength (nm)
    dirL=[]         #Direct Radiance (W/m^2/nm/sr)
    
    loops = int(len(data)/3)
    if phase =='Liquid Water': 
        dirL = [np.float(data[3*n+1][0][1:9]) for n in range(loops)]
        wl = [np.float(data[3*n+1][0][1:9]) for n in range(loops)]
    else:
        dirL = [np.float(data[3*n+2][0][9:24]) for n in range(loops)]
        wl = [np.float(data[3*n][0][1:9]) for n in range(loops)]

    dirLW = [i*10**-3 for i in dirL] #convert to W/m^2/nm/sr
    
    lrt_radiances = xr.DataArray(dirLW, dims=['wavelength'], name='LRT', coords={'wavelength':wl})
    lrt_radiances.attrs['COT'] = tau
    lrt_radiances.attrs['r_eff'] = reff
    return lrt_radiances;

path = 'WC files'
phase = 'Liquid Water'
files = [os.path.join(path,f) for f in os.listdir(path)]
file = files[0]
lrt_rad = read_LRT(file,phase)

In [9]:
def read_solar(file):
    d = np.loadtxt(file, delimiter="\t")
    solar_wl=d[:,0]
    solar_flux=d[:,1]
    solar_flux = np.array([i/1000 for i in solar_flux]) 
    solar = xr.DataArray(solar_flux,dims=['wavelength'], name='solar flux', coords={'wavelength':solar_wl})
    return solar;

solar_path = 'Aug14SolarRef.dat'                         
solar = read_solar(solar_path)                         

In [10]:
def filter_LRT_wl(lrt_rad):
    wl_min = 450 ; wl_max = 2300
    lrt_radf = lrt_rad.where(lrt_rad.wavelength>wl_min, drop=True)
    lrt_radf = lrt_radf.where(lrt_radf.wavelength<wl_max, drop=True)
    return lrt_radf;

def interpolate_solar(lrt_radf,solar):
    f = interp1d(solar.wavelength,solar.values,fill_value='NaN')
    solar_new = f(lrt_radf.wavelength)
    
    solar_interp = xr.DataArray(solar_new,dims=['wavelength'], name='interpolated solar flux', coords={'wavelength':lrt_radf.wavelength})
    return solar_interp;

def calc_LRT_reflectance(lrt_radf,solar_interp,phase):
    mu = np.cosd(38.5) if phase == 'Liquid Water' else np.cosd(56)
    refl_lrt = [np.pi/mu*lrt_radf.values[w]/solar_interp.values[w] for w in np.arange(0,len(lrt_radf.wavelength))]
                                                                        
    refl_lrt = xr.DataArray(refl_lrt,dims=['wavelength'], name='lrt reflectances', coords={'wavelength':lrt_radf.wavelength})
    refl_lrt.attrs['COT'] = lrt_radf.attrs['COT']
    refl_lrt.attrs['r_eff'] = lrt_radf.attrs['r_eff']                                                                                                                                      
    return refl_lrt;
                                                                   
#datacube_f = datacube_dvc.where(datacube_dvc.wavelength>400, drop=True)
lrt_radf = filter_LRT_wl(lrt_rad)
print(lrt_radf)
solar_interp = interpolate_solar(lrt_radf,solar)
print(solar_interp)
refl_lrt  = calc_LRT_reflectance(lrt_radf,solar_interp,phase)
print(refl_lrt)

<xarray.DataArray (wavelength: 617)>
array([0.4515, 0.4545, 0.4575, ..., 2.2935, 2.2965, 2.2995])
Coordinates:
  * wavelength  (wavelength) float64 451.5 454.5 457.5 ... 2.296e+03 2.3e+03
Attributes:
    COT:      1.529480
    r_eff:    37.5
<xarray.DataArray 'interpolated solar flux' (wavelength: 617)>
array([2.14  , 2.07  , 2.13  , ..., 0.0705, 0.0701, 0.0697])
Coordinates:
  * wavelength  (wavelength) float64 451.5 454.5 457.5 ... 2.296e+03 2.3e+03


AttributeError: module 'numpy' has no attribute 'cosd'

In [65]:
def calc_HySICS_reflectance(datacube,solar_interp,phase,x,y):    
    f=interp1d(datacube.wavelength,datacube.values[x,y,:],fill_value='NaN')
    hy_new = f(solar_interp.wavelength)
    
    mu = np.cosd(38.5) if phase == 'Liquid Water' else np.cosd(56)
    refl_HySICS= [np.pi/mu*hy_new[w]/solar_interp.values[w] for w in np.arange(0,len(solar_interp.wavelength))]
    
    refl_hysics_pixel = xr.DataArray(refl_HySICS,dims=['wavelength'], name='HySICS reflectance', coords={'wavelength':solar_interp.wavelength})
    return refl_hysics_pixel;

refl_hysics_pixel = calc_HySICS_reflectance(datacube_dvc,solar_interp,phase,0,0)

In [204]:
def find_retrieval_wavelengths(num_wl): 
    if num_wl%5 !=0: print('Number of wavelengths in algorithm must be a factor of 5.')   
    
    if (num_wl ==5):
        retrieval_wl_list=[[750], [1000], [1200], [1660], [2200]]
    else:
        wl_lims = [[600,750], [1000,1080], [1240,1320], [1600,1750], [2100,2200]]
        num = num_wl/5
        retrieval_wl_list = [np.linspace(wl_lims[w][0],wl_lims[w][1],num) for w in np.arange(0,5)]
    return retrieval_wl_list;

num_wl = 15
retrieval_wl_list = find_retrieval_wavelengths(num_wl)

/ncar/usr/jupyterhub/envs/pangeo-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  if __name__ == '__main__':


In [212]:
def find_retrieval_idx_list(retrieval_wl_list,solar_interp):   
    idx_list = []
    for i in np.arange(0,5):
        cols = []
        for j in np.arange(0,len(retrieval_wl_list[0])):
            a = [np.min(np.abs(retrieval_wl_list[i][j] - x)) for x in solar_interp.wavelength] 
            val, idx = min((val, idx) for (idx, val) in enumerate(a))
            cols.append(int(idx))
        idx_list.append(cols)
    return idx_list;

retrieval_idx_list = find_retrieval_idx_list(retrieval_wl_list,solar_interp)

In [225]:
def find_sigma_k_j(refl_lrt,refl_hysics,idx_1,k,j):
    reflectance_term = (5-k)**2*(refl_lrt.values[j]-refl_hysics.values[j])**2
    ratio_term = (k-1)**2*(refl_lrt.values[j]/refl_lrt.values[idx_1]- refl_hysics.values[j]/refl_hysics.values[idx_1])**2
    sigma_k_j = reflectance_term + ratio_term
    return sigma_k_j

def find_sigma_k(refl_lrt,refl_hysics,retrieval_idx_list,idx_1,i):
    k = i+1
    sigma_k = [find_sigma_k_j(refl_lrt,refl_hysics,idx_1,k,j) for j in retrieval_idx_list[i]] 
    return sigma_k

def calc_disagreement(refl_hysics,refl_lrt,retrieval_idx_list):  
    idx_1 = retrieval_idx_list[0][0]
    
    sigma_all = [find_sigma_k(refl_lrt,refl_hysics,retrieval_idx_list,idx_1,i) for i in np.arange(0,5)] 
    
    sigma = np.sum(sigma_all)
    chi_sq = sigma/(len(retrieval_idx_list[0])*60) 
    diff = np.sqrt(chi_sq)*100
    
    disagreement = xr.DataArray(diff)
    disagreement.attrs['COT'] = refl_lrt.attrs['COT']
    disagreement.attrs['r_eff'] = refl_lrt.attrs['r_eff']
    return disagreement;

disagreement = calc_disagreement(refl_hysics_pixel, refl_lrt, retrieval_idx_list)
disagreement.values

array(3571.77384153)

In [235]:
def gen_refl_lrt_list(path,phase,solar_interp,phase):
    refl_lrt_list = []
    files = [os.path.join(path,f) for f in os.listdir(path)]
    lrt_rads = [read_LRT(f,phase) for f in files if f.endswith('.dat')]
    lrt_radfs = [filter_LRT_wl(lrt_rad)for lrt_rad in lrt_rads]
    refl_lrts = [calc_LRT_reflectance(r,solar_interp,phase) for r in lrt_radfs]
    return (refl_lrts);

path = 'WC files'
phase = 'Liquid Water'
refl_lrt_list = gen_refl_lrt_list(path,phase,solar_interp,phase)

In [236]:
def calc_best_diff(refl_hysics_pixel,refl_lrt_list, retrieval_idx_list):    
    diff_list = [calc_disagreement(refl_hysics_pixel, r, retrieval_idx_list) for r in refl_lrt_list]
        
    best_diff, idx_diff = min((val, idx) for (idx, val) in enumerate(diff_list))
    return(best_diff);

best_diff = calc_best_diff(refl_hysics_pixel,refl_lrt_list, retrieval_idx_list)

In [241]:
best_diff.r_eff

'37.5'